In [3]:
import pandas as pd
import numpy as np
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [5]:
#input credentials
from config import *

In [77]:
def search_song(title:str, artist:str, limit=1) -> str:
    """
    Function takes two values: title and artist and then searches for it in Spotify
    It returns the number of results set by `limit`. 

    If the limit is great than 1, the user must select which item to pick.

    The function returns the Spotify id number.

    Input
    title: string of song title
    artist: string of artist name
    limit: the number of results to return

    Output
    string with Spotify song id number
    """
    
    #Initialize SpotiPy with user credentias #
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))


    #search for song
    results = sp.search(q="artist:" + artist + " track:" + title, limit=limit)

    print(len(results['tracks']['items']))

    if len(results['tracks']['items']) > 1:
        select_dict = {}
        for i in range(len(results['tracks']['items'])):
            #create a dataframe with the results
            id = results['tracks']['items'][i]['id']
            select_dict[id] = [results["tracks"]["items"][i]['name'],
                               results['tracks']['items'][i]['album']['artists'][0]['name'],
                               results['tracks']['items'][i]['album']['name']]
            
        df_results = (pd.DataFrame.from_dict(select_dict,
                                             orient='index',
                                             columns = ['title', 'artist', 'album'])
                      .reset_index(names = 'id',
                                   drop=False)
                      .drop_duplicates(subset = ['title', 'artist', 'album'],
                                       keep='first')
                     )
        
        display(df_results[['title', 'artist', 'album']])
            
        row_number = int(input('Please enter the number of the song you are looking for'))

        id_number = [df_results.iloc[row_number]['id']]

        return id_number


In [78]:
search_song('Bad Blood', 'Taylor Swift', 3)

3


,id,title,artist,album
0,273dCMFseLcVsoSWx59IoE,Bad Blood,Taylor Swift,1989
1,6xsEAm6w9oMQYYg3jkEkMT,Bad Blood,Taylor Swift,Bad Blood
2,0yN4fNzmVnmgC0dsOoi9Wh,Bad Blood,Taylor Swift,1989 (Deluxe Edition)


,title,artist,album
0,Bad Blood,Taylor Swift,1989
1,Bad Blood,Taylor Swift,Bad Blood
2,Bad Blood,Taylor Swift,1989 (Deluxe Edition)


Please enter the number of the song you are looking for 0


['273dCMFseLcVsoSWx59IoE']

In [109]:
def get_audio_features(list_of_ids)->pd.DataFrame:
    """
    This function takes a list of id values and gets the spotify audio features for this song.
    The function returns a pandas DataFrame with all the information about the song, including title and artist

    Input
    list_of_ids: a list with id values as strings

    Output
    pandas DataFrame with the title of the song, name of the artist and all features
    """
    
    #Initialize SpotiPy with user credentias #
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))
    full_dict={}
    for id in list_of_ids:
        
        my_dict = sp.audio_features(id)[0]

        #print(my_dict)

        my_dict_new = { key : [my_dict[key]] for key in list(my_dict.keys()) }
        #print(my_dict_new)
        
        full_dict[id] = my_dict_new
    
    #print(full_dict)
        #my_dict_new['titles'] = [results["tracks"]["items"][0]['name']]
        #my_dict_new['artists']= [results['tracks']['items'][0]['album']['artists'][0]['name']]
        
        
    features_df = pd.DataFrame.from_dict(full_dict,orient='index')

    return features_df
    

In [110]:
test_list =['3k79jB4aGmMDUQzEwa46Rz','273dCMFseLcVsoSWx59IoE']

In [111]:
get_audio_features(test_list)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
3k79jB4aGmMDUQzEwa46Rz,[0.511],[0.532],[5],[-5.745],[1],[0.056],[0.169],[0],[0.311],[0.322],[137.827],[audio_features],[3k79jB4aGmMDUQzEwa46Rz],[spotify:track:3k79jB4aGmMDUQzEwa46Rz],[https://api.spotify.com/v1/tracks/3k79jB4aGmM...,[https://api.spotify.com/v1/audio-analysis/3k7...,[219724],[4]
273dCMFseLcVsoSWx59IoE,[0.65],[0.802],[7],[-6.114],[1],[0.181],[0.0871],[6.54e-06],[0.148],[0.295],[170.146],[audio_features],[273dCMFseLcVsoSWx59IoE],[spotify:track:273dCMFseLcVsoSWx59IoE],[https://api.spotify.com/v1/tracks/273dCMFseLc...,[https://api.spotify.com/v1/audio-analysis/273...,[211933],[4]
